# Auto AudioBooks



Preparing the environment


OCR for Optical Character Recognizition on Input PDF Books/Papers

In [1]:
!pip install pytesseract
!sudo apt install tesseract-ocr


Text to Speech for creating Audiobooks

In [ ]:
!pip install gTTS
!pip install pdf2image

In [ ]:

!apt-get install -y poppler-utils


In [5]:
import pytesseract 
from gtts import gTTS
from IPython.display import Audio
from PIL import Image 
from pdf2image import convert_from_path 
import os


Utility to create DIR

In [6]:
def createDir(dirName):
  if not os.path.exists(dirName):
    os.makedirs(dirName)
    print("Directory " , dirName ,  " Created ")
  else:    
    print("Directory " , dirName ,  " already exists")


Utility to extract filename

In [7]:
def get_filename_without_extension(file_path):
    file_basename = os.path.basename(file_path)
    filename_without_extension = file_basename.split('.')[0]
    return filename_without_extension

In [8]:
def percentage(curr,total):
  return curr/total * 100;

Constants

In [9]:
IMAGES_DIR = "/content/Images/"
AUDIO_DIR = "/content/Audio/"
createDir(IMAGES_DIR)
createDir(AUDIO_DIR)

Directory  /content/Images/  Created 
Directory  /content/Audio/  Created 


Convert PDF to image snapshots

In [31]:
def pdf2Image(fn):
  file = get_filename_without_extension(fn)
  image_fn_dir= IMAGES_DIR + file
  createDir(image_fn_dir)
  print(fn);
  pages = convert_from_path(fn, 500)
  print(pages)
  filenames = [];
  
  # Counter to store images of each page of PDF to image 
  image_counter = 1
  total = len(pages)

  # Iterate through all the pages stored above 
  for page in pages: 
    
      # Declaring filename for each page of PDF as JPG 
      # For each page, filename will be: 
      # PDF page 1 -> page_1.jpg 
      # PDF page 2 -> page_2.jpg 
      # PDF page 3 -> page_3.jpg 
      # .... 
      # PDF page n -> page_n.jpg 
      filename = image_fn_dir+ "/page_"+str(image_counter)+".jpg"
      filenames.append(filename);
      # Save the image of the page in system 
      page.save(filename, 'JPEG') 
      
      print("Conversion - "+str(percentage(image_counter,total)) +" % work done yay!")

      # Increment the counter to update filename 
      image_counter = image_counter + 1
      
  return filenames

Convert Images to Text using pytesseract OCR

In [14]:
def convertImagesToText(filenames, ofilename):
  # Creating a text file to write the output 
  outfile = "out_text_"+ofilename+".txt"
  
  # Open the file in append mode so that  
  # All contents of all images are added to the same file 
  f = open(outfile, "a")
  for file in filenames:
    print(file)
    text = str(((pytesseract.image_to_string(Image.open(file)))))
    text = text.replace('-\n', '')     
    text = text + '\n'
    f.write(text)
  f.close()
  print(outfile + " prepared :) ")
  return outfile

Upload pdf to be converted to AudioBook ^_^

In [30]:
from google.colab import files
uploaded = files.upload()

Saving research-paper.pdf to research-paper.pdf
Saving sample.pdf to sample.pdf


Given all input pdfs, convert them to audio books

In [32]:
for fn in uploaded.keys():
  # predict images
  filename = get_filename_without_extension(fn)
  path = "/content/" + fn
  filenames = pdf2Image(path);
  outFile = convertImagesToText(filenames,filename);
  text = open(outFile).read()
  ta_tts = gTTS(text)
  ta_tts.save(AUDIO_DIR + filename + '.mp3')

Directory  /content/Images/research-paper  Created 
/content/research-paper.pdf
[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4250x5500 at 0x7F203BCA8160>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4250x5500 at 0x7F203BCA8278>]
Conversion - 50.0 % work done yay!
Conversion - 100.0 % work done yay!
/content/Images/research-paper/page_1.jpg
/content/Images/research-paper/page_2.jpg
out_text_research-paper.txt prepared :) 
Directory  /content/Images/sample  already exists
/content/sample.pdf
[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4250x5500 at 0x7F203BCA8588>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=4250x5500 at 0x7F203BCA8748>]
Conversion - 50.0 % work done yay!
Conversion - 100.0 % work done yay!
/content/Images/sample/page_1.jpg
/content/Images/sample/page_2.jpg
out_text_sample.txt prepared :) 


Preview first generated audiobook

In [33]:
preview = get_filename_without_extension(next(iter(uploaded)))

In [34]:
audio_file_preview = AUDIO_DIR+preview+".mp3"
print(audio_file_preview)

/content/Audio/research-paper.mp3


In [35]:
Audio(audio_file_preview, autoplay=True)